## Data Analysis for the paper "Revolutionizing Horizontal Gene Transfer Detection"

In [1]:
import numpy as np
import pandas as pd
import altair as alt
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score
import scipy.stats as stats
import statsmodels.api as sm

In [2]:
#parameters for chart's display
font_size = 13

In [3]:
#reference: https://github.com/AAnzel/TVSDS/blob/master/Source/UI.py
#a function to determine the location of labels in the middle of two data points
def calc_midpoints(y):
    x = []
    for i in range(len(y)):
        prev = y[: i]
        x.append(y[i]/2 + sum(prev))

    return x

In [4]:
#Read data
df = pd.read_excel("Data/hgt_summary_altair.xlsx", sheet_name="methods")

In [5]:
df.head()

,Year,Model,Link,Method,Assumption,Dataset,Metrics,Results,Limitation,Dataset Availability,...,Features,Complete or Draft genome,Data Type,Target,Algorithm,Algorithm Type,Mechanism or Consequence,Computational Group,Citations,Additional Notes
0,2000,"Garcia-Vallvé, Santiago, et al.",https://genome.cshlp.org/content/10/11/1719.short,- a statistical procedure for predicting wheth...,- a reliable HGT prediction must include genes...,"- 17 bacterial genomes, 7 archaeal genomes wit...",NaN,- informational genes were less likely to be ...,- genes shorter than 300 bp are excluded,GenBank (real),...,"GC content, codon usage, nucleotide, gene posi...",complete,WGS,gene,NaN,NaN,consequence,Sequence Composition,274,NaN
1,2001,Pyphy,A phylogenomic approach to microbial evolution...,- discrepancies between a species trees and a ...,- transferred genes are positioned deeply with...,- 7 microbial genomes,NaN,NaN,- Phylogenetic methods rely heavily on the acc...,KEGG dataset \nperformance not reported,...,gene,complete,WGS,gene,NaN,NaN,consequence,Comparative Genomics,130,NaN
2,2001,LatTrans,Efficient algorithms for lateral gene transfer...,- discrepancies between a species trees and a ...,NaN,- 48 rbcL (the gene for the large subunit of r...,NaN,NaN,NaN,https://biology.indiana.edu/-jpalmer/rubisco-e...,...,gene,complete,WGS,gene,NaN,NaN,consequence,Comparative Genomics,101,NaN
3,2001,Self-Organizing Map (SOM),https://www.sciencedirect.com/science/article/...,- apply SOM to characterize codon usage hetero...,- Genes introduced through horizontal transfer...,- 29 bacterial species containing ~60k genes w...,NaN,NaN,NaN,GenBank (real),...,codon usage,complete,WGS,gene,SOM,Clustering,consequence,Artificial Intelligence (AI),157,NaN
4,2002,Genomic 3:1 Signature,Detection of genes with atypical nucleotide se...,- measure the composition bias of each gene w....,- genes from different genomes can be separate...,- 10 bacterial genomes,NaN,NaN,- limited to recent HGT before assimilation oc...,NCBI (real data),...,dinucleotide,"draft, complete",WGS,gene,NaN,NaN,consequence,Sequence Composition,32,NaN


In [6]:
#Assign label of each approach whether it is machine learning (ML), deep learning (DL) or others 
df["Approach"] = df["Processes"].astype(object).apply(lambda x: 'Machine Learning (ML)' if "machine" in x and "deep" not in x else ('Deep Learning (DL)' if "deep" in x else "Others"))

## General Trend of Computational Approaches for HGT Detection

In [7]:
overall_trend = df[["Year"]].groupby(["Year"])["Year"].count().cumsum() 
overall_trend_df = overall_trend.to_frame().rename(columns={"Year":"Cumulative Count"}).reset_index()

In [8]:
#plot the trend of all approaches for HGT detection
overall_yearly_trend = alt.Chart(overall_trend_df).mark_bar(stroke='gray').encode(
    x=alt.X('Year:O', axis=alt.Axis(labelAngle=-45)),
    y="Cumulative Count",
).properties(width=600)

overall_yearly_trend

alt.Chart(...)

## Trend of Machine Learning (ML) and Deep Learning (DL) in the field of HGT Detection

In [9]:
#trend of ML and DL applications in computational approaches for Horizontal Gene Transfer (HGT)
ml_dl_trend = df[["Year","Approach"]].groupby(["Year","Approach"])["Approach"].count()\
                                                                            .unstack()\
                                                                            .reset_index()\
                                                                            .sort_values(by=['Year'], ascending=True)\
                                                                            .set_index('Year')\
                                                                            .fillna(0)\
                                                                            .cumsum()

ml_dl_trend_new = ml_dl_trend.stack().reset_index().rename(columns={0:"Cumulative Count"})\
                            .sort_values(by=['Year','Approach'], ascending=[True, False])

ml_dl_trend_new.head()

,Year,Approach,Cumulative Count
2,2000,Others,1.0
1,2000,Machine Learning (ML),0.0
0,2000,Deep Learning (DL),0.0
5,2001,Others,3.0
4,2001,Machine Learning (ML),1.0


In [10]:
#plot the trend of ML and DL applications in HGT detection
ml_dl_yearly_trend = alt.Chart(ml_dl_trend_new).mark_bar(stroke='gray').encode(
    x=alt.X('Year:O', axis=alt.Axis(labelAngle=-45)),
    y="Cumulative Count",
    color=alt.Color('Approach', 
                    scale=alt.Scale(domain=["Deep Learning (DL)","Machine Learning (ML)","Others"],
                                    range=['#A2A2A2','#555555','#FFFFFF']),
                   legend=alt.Legend(orient='none',
                                    legendX=100, legendY=360,
                                    direction='horizontal',
                                    title='',
                                    labelPadding=10.0,
                                    labelOffset=1.0,
                                    columnPadding=25.0)),
    order=alt.Order("Approach").sort("ascending")
).properties(width=600)

font_size = 13
ml_dl_yearly_trend.configure_axis(
    labelFontSize=font_size,
    titleFontSize=font_size
).configure_legend(
    labelFontSize=font_size
)

/Users/miko/anaconda3/envs/viz/lib/python3.11/site-packages/altair/utils/core.py:410: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

Existing computational approaches can be categorized into 4 groups, namely:
1. artificial intelligence (AI)
2. sequence composition
3. comparative genomics
4. hybrid

## Trend of Computational Groups of HGT Detection

In [11]:
#process the table to calculate the cumulative sum of each computational group per year
computational_group_trend = df[["Year","Computational Group"]].groupby(["Year","Computational Group"])["Computational Group"].count()\
                                                                                                                            .unstack()\
                                                                                                                            .reset_index()\
                                                                                                                            .sort_values(by=['Year'], ascending=True)\
                                                                                                                            .set_index('Year')\
                                                                                                                            .fillna(0)\
                                                                                                                            .cumsum()

#collapse the column into rows for creating a stacked bar chart
computational_group_trend_stack = computational_group_trend.stack().reset_index().rename(columns={0:"Cumulative Count"})

In [12]:
computational_group_trend_stack.head()

,Year,Computational Group,Cumulative Count
0,2000,Artificial Intelligence (AI),0.0
1,2000,Comparative Genomics,0.0
2,2000,Hybrid,0.0
3,2000,Sequence Composition,1.0
4,2001,Artificial Intelligence (AI),1.0


In [17]:
#plot the trend of computational groups for HGT detection
computational_group_trend_chart = alt.Chart(computational_group_trend_stack).mark_bar(stroke='gray').encode(
    x=alt.X('Year:O', axis=alt.Axis(labelAngle=-45)),
    y="Cumulative Count",
    color=alt.Color('Computational Group', 
                                        scale=alt.Scale(domain=["Artificial Intelligence (AI)","Comparative Genomics","Hybrid","Sequence Composition"],
                                                        range=['#000000','#a6cee3','#1f78b4','#b2df8a']),
                   legend=alt.Legend(orient='none',
                                                                            legendX=10, legendY=430,
                                                                            direction='horizontal',
                                                                            title='',
                                                                            labelPadding=10.0,
                                                                            labelOffset=1.0,
                                                                            columnPadding=25.0)),
    order=alt.Order("Computational Group").sort("ascending")
).properties(width=600)

computational_group_trend_chart

/Users/miko/anaconda3/envs/viz/lib/python3.11/site-packages/altair/utils/core.py:410: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [18]:
#plot of the proportion of each computational group at the latest state (year 2023)

#take the latest year (2023) to show the current state of the computational groups for HGT detection
latest_state = computational_group_trend_stack[computational_group_trend_stack['Year']==2023]

#calculate the proportion of each computational group in the latest year
total_computational_approaches = latest_state.loc[:,'Cumulative Count'].values.sum()
latest_state.loc[:,'proportion'] = latest_state['Cumulative Count']/total_computational_approaches
latest_state.loc[:,'text_pos'] = calc_midpoints(latest_state.loc[:,'proportion'].values)

#split the label color into white and black color for better visibility
white_font = latest_state[latest_state['Computational Group'].str.contains("AI|Hybrid")]
black_font = latest_state[latest_state['Computational Group'].str.contains("Sequence|Comparative")]

computational_trend_proportion_bar = alt.Chart(latest_state).mark_bar(size=20, stroke='gray').encode(
    x=alt.X('sum(proportion)',axis=alt.Axis(labels=False, tickSize=0)).title('Proportion').stack('normalize'),
    color=alt.Color('Computational Group', 
                    scale=alt.Scale(domain=["Artificial Intelligence (AI)","Comparative Genomics","Hybrid","Sequence Composition"],
                                    range=['#000000','#a6cee3','#1f78b4','#b2df8a']))
)

white_text = alt.Chart(white_font).mark_text(size=12, color='#FAF9F6').encode(
    x=alt.X('text_pos:Q'),
    detail='Computational Group',
    text=alt.Text('proportion:Q', format=".2%")
)

black_text = alt.Chart(black_font).mark_text(size=12, color='#363636').encode(
    x=alt.X('text_pos:Q'),
    detail='Computational Group',
    text=alt.Text('proportion:Q', format=".2%")
)

computational_trend_proportion_bar_w_text = (computational_trend_proportion_bar+white_text+black_text).properties(width=600)
computational_trend_proportion_bar_w_text

/var/folders/wz/b54c30xd2nvfn7l1_bqy57yr0000gn/T/ipykernel_3380/231016143.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  latest_state.loc[:,'proportion'] = latest_state['Cumulative Count']/total_computational_approaches
/var/folders/wz/b54c30xd2nvfn7l1_bqy57yr0000gn/T/ipykernel_3380/231016143.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  latest_state.loc[:,'text_pos'] = calc_midpoints(latest_state.loc[:,'proportion'].values)
/Users/miko/anaconda3/envs/viz/lib/python3.11/site-packages/altair/utils

alt.LayerChart(...)

In [19]:
#final plot of the trend of computational approaches per group between 2000 and 2023
computational_group_trend_final = alt.vconcat(computational_group_trend_chart, computational_trend_proportion_bar_w_text)
computational_group_trend_final

computational_group_trend_final.configure_axis(
    labelFontSize=font_size,
    titleFontSize=font_size
).configure_legend(
    labelFontSize=font_size
)

/Users/miko/anaconda3/envs/viz/lib/python3.11/site-packages/altair/utils/core.py:410: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/miko/anaconda3/envs/viz/lib/python3.11/site-packages/altair/utils/core.py:410: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/miko/anaconda3/envs/viz/lib/python3.11/site-packages/altair/utils/core.py:410: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/mik

alt.VConcatChart(...)

## Analysis of Data Type used in the Computational Approaches

In [20]:
#trend of data type used in computational approaches 
data_type_trend = df[["Year","Data Type"]].groupby(["Year","Data Type"])["Data Type"].count()\
                                                                            .unstack()\
                                                                            .reset_index()\
                                                                            .sort_values(by=['Year'], ascending=True)\
                                                                            .set_index('Year')\
                                                                            .fillna(0)\
                                                                            .cumsum()

data_type_stack = data_type_trend.stack().reset_index().rename(columns={0:"Cumulative Count"})\
                            .sort_values(by=['Year','Data Type'], ascending=[True, False])

In [21]:
#plot the data type used in all computational approaches 

data_type_trend_chart = alt.Chart(data_type_stack).mark_bar(stroke='gray').encode(
    x=alt.X('Year:O', axis=alt.Axis(labelAngle=-45)),
    y="Cumulative Count",
    color=alt.Color('Data Type', 
                    scale=alt.Scale(domain=["Metagenome","WGS","WGSS"],
                                    range=['#fc8d62', '#66c2a5', '#8da0cb']),
                   legend=alt.Legend(orient='none',
                                    legendX=190, legendY=350,
                                    direction='horizontal',
                                    title='',
                                    labelPadding=10.0,
                                    labelOffset=1.0,
                                    columnPadding=25.0)),
    order=alt.Order("Data Type").sort("ascending")
).properties(width=600)

data_type_trend_chart

/Users/miko/anaconda3/envs/viz/lib/python3.11/site-packages/altair/utils/core.py:410: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [22]:
#trend of data type used in each computational group 
data_type_per_group = df[["Computational Group","Data Type"]].groupby(["Computational Group","Data Type"])["Data Type"].count()\
                                                                            .unstack()\
                                                                            .reset_index()\
                                                                            .set_index('Computational Group')\
                                                                            .fillna(0)
                                                                            #.rename(index={"artificial intelligence (AI)":1,
                                                                            #              "sequence composition":2,
                                                                            #              "comparative genomics":3,
                                                                            #              "hybrid":4})
#collapse the column into rows for creating a stacked bar chart
data_type_per_group_stack = data_type_per_group.stack().reset_index().rename(columns={0:"Proportion"}).sort_values(by=['Computational Group'])

#plot stack bar chart of data type (color scheme= #66c2a5, #fc8d62, #8da0cb)
data_trend_per_group_chart = alt.Chart(data_type_per_group_stack).mark_bar().encode(
    y=alt.Y('Computational Group:O', axis=alt.Axis(title="", titlePadding=20)),
    x=alt.X('Proportion', axis=alt.Axis(tickCount=10, grid=False, labelExpr="datum.value*100 % 20 ? null : datum.label")).stack("normalize"),
    color=alt.Color('Data Type', 
                    scale=alt.Scale(domain=["Metagenome", "WGS","WGSS"],
                                    range=['#fc8d62', '#66c2a5', '#8da0cb']),
                   legend=alt.Legend(orient='none',
                                    legendX=70, legendY=200,
                                    direction='horizontal',
                                    title='',
                                    labelPadding=10.0,
                                    labelOffset=1.0,
                                    columnPadding=25.0)),
    order=alt.Order("Data Type").sort("ascending")
).properties(height=150,width=400)

data_trend_per_group_chart.configure_axis(
    labelFontSize=font_size,
    titleFontSize=font_size
).configure_legend(
    labelFontSize=font_size
)

/Users/miko/anaconda3/envs/viz/lib/python3.11/site-packages/altair/utils/core.py:410: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

## Trend Analysis with Linear and Exponential Regression Models

In [23]:
#analysis of overall trend using linear and exponential regression models
x = overall_trend.index
y = overall_trend.values

correlation = np.corrcoef(x, y)[0,1]

lin_fn = np.polyfit(x, overall_trend.values, 1)
exp_fn = np.polyfit(x, overall_trend.values, 2)
lin_fit = np.poly1d(lin_fn)
exp_fit = np.poly1d(exp_fn)

slope, r2_lin, r2_exp = lin_fit.coef[0],r2_score(y, lin_fit(x)),r2_score(y, exp_fit(x))

overall_trend_df_renamed = overall_trend_df.rename(columns={'Cumulative Count':'original_data'})

#insert regression line into dataframe with original data points
overall_trend_df_renamed['linear_line'] = lin_fit(x)
overall_trend_df_renamed['exp_line'] = exp_fit(x)

#unstack the columns for visualization purpose in the following steps
overall_trend_df_w_regression = overall_trend_df_renamed.set_index('Year')\
                                                        .unstack()\
                                                        .reset_index()\
                                                        .rename(columns={0:'Cumulative Count',
                                                                         'level_0':'fitting_line'})

#plot layered chart to show how the original data points fit linear and exponential regression lines

original_data = alt.Chart(overall_trend_df_w_regression).mark_circle().transform_filter(
    alt.datum.fitting_line == "original_data"
).encode(
    x=alt.X('Year:O', axis=alt.Axis(labelAngle=-45)),
    y='Cumulative Count:Q',
)

regression_line = alt.Chart(overall_trend_df_w_regression).mark_line().transform_filter(
    alt.datum.fitting_line != "original_data"
).encode(
    x=alt.X('Year:O', axis=alt.Axis(labelAngle=-45)),
    y='Cumulative Count:Q',
    color=alt.Color('fitting_line:N',
                    scale=alt.Scale(range=['red','green']))
)

group_name = alt.Chart({'values':[{'Year': 2012, 'Cumulative Count': 135}]}).mark_text(
        text='All Computational Approaches',
    ).encode(
        x='Year:O', y='Cumulative Count:Q'
    )
    
stats_overall = alt.Chart({'values':[{'Year': 2012, 'Cumulative Count': 128}]}).mark_text(
    text='(Slope={:.3f}, R\u00b2 linear= {:.3f}, R\u00b2 exp={:.3f})'.format(slope,r2_lin,r2_exp),
).encode(
    x='Year:O', y='Cumulative Count:Q'
)

final_chart_overall = alt.layer(original_data, regression_line, group_name, stats_overall).configure_axis(
    labelFontSize=font_size,
    titleFontSize=font_size
).configure_legend(
    labelFontSize=font_size
)

final_chart_overall

/Users/miko/anaconda3/envs/viz/lib/python3.11/site-packages/altair/utils/core.py:410: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/miko/anaconda3/envs/viz/lib/python3.11/site-packages/altair/utils/core.py:410: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.LayerChart(...)

In [24]:
#analysis of overall trend using linear and exponential regression models

groups = computational_group_trend.columns.values

regression_test_df = pd.DataFrame()

hconcat = alt.hconcat()
vconcat = alt.vconcat()

for i,group in enumerate(groups):
    x = computational_group_trend[group].index.values
    y = computational_group_trend[group].values

    correlation = np.corrcoef(x, y)[0,1]

    lin_fn = np.polyfit(x, computational_group_trend[group].values, 1)
    exp_fn = np.polyfit(x, computational_group_trend[group].values, 2)
    lin_fit = np.poly1d(lin_fn)
    exp_fit = np.poly1d(exp_fn)
    
    group_df = computational_group_trend[group].reset_index()\
                                               .rename(columns={group:'original_data'})

    #insert regression line into dataframe of original data points
    group_df['linear_line'] = lin_fit(x)
    group_df['exp_line'] = exp_fit(x) 

    #unstack the columns for visualization purpose in the following steps
    group_df_w_regression = group_df.set_index('Year').unstack()\
                                                      .reset_index()\
                                                      .rename(columns={0:'Cumulative Count',
                                                                       'level_0':'fitting_line'})
    
    slope, r2_lin, r2_exp = lin_fit.coef[0],r2_score(y, lin_fit(x)),r2_score(y, exp_fit(x))
    regression_test_df_temp = pd.DataFrame([[group,slope,r2_lin,r2_exp]], 
                                           columns=['Group','Slope','R2_lin_model','R2_exp_model'])
    regression_test_df = pd.concat([regression_test_df, regression_test_df_temp])
    
    #plot layered chart to show how the original data points fit linear and exponential regression lines
    original_data = alt.Chart(group_df_w_regression[group_df_w_regression['fitting_line']=='original_data']).mark_circle().encode(
        x=alt.X('Year:O', axis=alt.Axis(labelAngle=-45)),
        y='Cumulative Count:Q',
    )

    regression_line = alt.Chart(group_df_w_regression[group_df_w_regression['fitting_line']!='original_data']).mark_line().encode(
        x=alt.X('Year:O', axis=alt.Axis(labelAngle=-45)),
        y=alt.Y('Cumulative Count:Q').scale(domain=(-10, 60)),
        color=alt.Color('fitting_line:N',
                        scale=alt.Scale(range=['red','green']))
    )
    
    group_name = alt.Chart({'values':[{'Year': 2012, 'Cumulative Count': 58}]}).mark_text(
        text=group,
    ).encode(
        x='Year:O', y='Cumulative Count:Q'
    )
    
    stats_group = alt.Chart({'values':[{'Year': 2012, 'Cumulative Count': 55}]}).mark_text(
        text='(Slope={:.3f}, R\u00b2 linear= {:.3f}, R\u00b2 exp={:.3f})'.format(slope,r2_lin,r2_exp),
    ).encode(
        x='Year:O', y='Cumulative Count:Q'
    )

    chart = alt.layer(original_data, regression_line, group_name, stats_group).properties(height=300,width=350)
    
    if i%2==1:
        hconcat |= chart
        vconcat &= hconcat
        hconcat = alt.hconcat()
    else:
        hconcat |= chart

font_size=10
vconcat.configure_axis(
    labelFontSize=font_size,
    titleFontSize=font_size,
).configure_legend(
    labelFontSize=font_size
)

/Users/miko/anaconda3/envs/viz/lib/python3.11/site-packages/altair/utils/core.py:410: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/miko/anaconda3/envs/viz/lib/python3.11/site-packages/altair/utils/core.py:410: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/miko/anaconda3/envs/viz/lib/python3.11/site-packages/altair/utils/core.py:410: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/mik

alt.VConcatChart(...)

## t-test between Pairs of Computational Groups

In [25]:
stats_test_df = pd.DataFrame()

for g1 in groups:
    for g2 in groups:
        if g1==g2:
            continue
        s1 = computational_group_trend[g1]
        s2 = computational_group_trend[g2]
        corr = s1.corr(s2)
        
        # t-test between computational groups
        t_statistic, p_value = stats.ttest_rel(s1, s2)
       
        reject = p_value<0.05
        res = pd.DataFrame([[g1,g2,corr,t_statistic,p_value,reject]], columns=['group_1','group_2','correlation','t_statistic','p_value','reject_null_hypothesis'])
        
        stats_test_df = pd.concat([stats_test_df, res])
        
stats_test_df

,group_1,group_2,correlation,t_statistic,p_value,reject_null_hypothesis
0,Artificial Intelligence (AI),Comparative Genomics,0.983038,-6.505897,0.000001,True
0,Artificial Intelligence (AI),Hybrid,0.972413,-6.485148,0.000001,True
0,Artificial Intelligence (AI),Sequence Composition,0.879457,-6.328486,0.000002,True
0,Comparative Genomics,Artificial Intelligence (AI),0.983038,6.505897,0.000001,True
0,Comparative Genomics,Hybrid,0.980034,5.641046,0.000010,True
0,Comparative Genomics,Sequence Composition,0.901642,3.779972,0.000970,True
0,Hybrid,Artificial Intelligence (AI),0.972413,6.485148,0.000001,True
0,Hybrid,Comparative Genomics,0.980034,-5.641046,0.000010,True
0,Hybrid,Sequence Composition,0.942707,-3.103609,0.005005,True
0,Sequence Composition,Artificial Intelligence (AI),0.879457,6.328486,0.000002,True


## Significance Test of the Slope of the Trend

In [28]:
import statsmodels.api as sm

group = 'Artificial Intelligence (AI)'
df1 = computational_group_trend[group].reset_index().rename(columns={'Artificial Intelligence (AI)':'Y'})
df1['X'] = df1['Year'].apply(lambda x: x%2000 + 1)

group = 'Sequence Composition'
df2 = computational_group_trend[group].reset_index().rename(columns={'Sequence Composition':'Y'})
df2['X'] = df2['Year'].apply(lambda x: x%2000 + 1)

group = 'Comparative Genomics'
df3 = computational_group_trend[group].reset_index().rename(columns={'Comparative Genomics':'Y'})
df3['X'] = df3['Year'].apply(lambda x: x%2000 + 1)

group = 'Hybrid'
df4 = computational_group_trend[group].reset_index().rename(columns={'Hybrid':'Y'})
df4['X'] = df4['Year'].apply(lambda x: x%2000 + 1)

result1 = sm.OLS(df1['Y'], sm.add_constant(df1['X'])).fit()
result2 = sm.OLS(df2['Y'], sm.add_constant(df2['X'])).fit()
result3 = sm.OLS(df3['Y'], sm.add_constant(df3['X'])).fit()
result4 = sm.OLS(df4['Y'], sm.add_constant(df4['X'])).fit()

print('Artificial Intelligence (AI) \n',result1.summary())
print('\n Sequence Composition \n',result2.summary())
print('\n Comparative Genomics \n',result3.summary())
print('\n Hybrid \n',result4.summary())

Artificial Intelligence (AI) 
                             OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.916
Model:                            OLS   Adj. R-squared:                  0.913
Method:                 Least Squares   F-statistic:                     241.2
Date:                Wed, 10 Apr 2024   Prob (F-statistic):           2.43e-13
Time:                        12:30:25   Log-Likelihood:                -51.064
No. Observations:                  24   AIC:                             106.1
Df Residuals:                      22   BIC:                             108.5
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -4.1051

In [29]:
import statsmodels.formula.api as smf

# Combine all datasets into one DataFrame with an indicator variable for each dataset
combined_df = pd.concat([df2.assign(group=2), df2.assign(group=2),df3.assign(group=3), df4.assign(group=4)])

# Fit a regression model with interaction term
model = smf.ols(formula='Y ~ X * group', data=combined_df)
result = model.fit()

print("******************** Comparison of all computational groups ******************** \n\n",result.summary())

******************** Comparison of all computational groups ******************** 

                             OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.709
Model:                            OLS   Adj. R-squared:                  0.700
Method:                 Least Squares   F-statistic:                     74.75
Date:                Wed, 10 Apr 2024   Prob (F-statistic):           1.40e-24
Time:                        12:30:28   Log-Likelihood:                -306.97
No. Observations:                  96   AIC:                             621.9
Df Residuals:                      92   BIC:                             632.2
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------

## ANOVA Test

In [30]:
# Example data
group1 = df1['Y'].values
group2 = df2['Y'].values
group3 = df3['Y'].values
group4 = df4['Y'].values

# Perform one-way ANOVA
f_statistic, p_value = stats.f_oneway(group1, group2, group4)

# Print results
print("ANOVA test without comparative genomics")
print("F-statistic:", f_statistic)
print("P-value:", p_value)

# Check for significance
alpha = 0.05
if p_value < alpha:
    print("Reject the null hypothesis; there are significant differences.")
else:
    print("Fail to reject the null hypothesis; no significant differences.")
    
# Perform one-way ANOVA
f_statistic, p_value = stats.f_oneway(group1, group2, group3, group4)

# Print results
print("\n ANOVA test with comparative genomics")
print("F-statistic:", f_statistic)
print("P-value:", p_value)

# Check for significance
alpha = 0.05
if p_value < alpha:
    print("Reject the null hypothesis; there are significant differences.")
else:
    print("Fail to reject the null hypothesis; no significant differences.")


ANOVA test without comparative genomics
F-statistic: 2.2178844661270656
P-value: 0.11654129002433479
Fail to reject the null hypothesis; no significant differences.

 ANOVA test with comparative genomics
F-statistic: 6.762207058598525
P-value: 0.00035856428231118974
Reject the null hypothesis; there are significant differences.


In [24]:
, also referred to intra-attention~\cite{cheng2016long},

('also', 'referred', 'to', 'intra-attention~\\cite{cheng2016long},')